## Setup for google colab

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
%cd /content/drive/My Drive/SA/Code/spoof_detection_deep_features/WaveletCNN

/content/drive/My Drive/SA/Code/spoof_detection_deep_features/WaveletCNN


## Run code from here

In [1]:
!pip install keras==2.0.8

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
  Using cached Keras-2.0.8-py2.py3-none-any.whl (276 kB)
  Attempting uninstall: keras
    Found existing installation: Keras 2.2.4
    Uninstalling Keras-2.2.4:
      Successfully uninstalled Keras-2.2.4


In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="0";  

In [2]:
import tensorflow as tf
sess = tf.Session()

from keras import backend as K
K.set_session(sess)

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
import warnings
warnings.simplefilter(action='ignore', category=Warning)
import numpy as np
import librosa as lb
from tensorflow import keras
import tensorflow as tf

from matplotlib import pyplot as plt
from keras.optimizers import Adam,SGD
from keras.backend.tensorflow_backend import set_session
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import Lambda
from keras.layers import Flatten
from keras.layers import Reshape
from keras.layers import Dropout
from keras.layers import Activation
from keras.layers import AveragePooling1D
from keras.layers import BatchNormalization
from keras.layers.merge import add, concatenate
from keras.utils import plot_model
from keras.utils import to_categorical,Sequence
import pandas as pd
import math
import sincnet
# from tensorflow import set_random_seed
from keras import models, layers
import numpy as np
import sincnet
from keras.layers import Dense, Dropout, Activation
from keras.layers import MaxPooling1D,MaxPooling2D, Conv1D, LeakyReLU, BatchNormalization, Dense, Flatten
from keras.layers import InputLayer, Input
from keras.models import Model
# from tensorflow.python.keras.utils.data_utils import Sequence
from sklearn.preprocessing import StandardScaler
import pywt
from sklearn.decomposition import PCA
import sys
import tensorflow.contrib.slim as slim
# import tfwavelets
# sys.path.insert(1, '/content/drive/My Drive/SA/Code/spoof_detection_deep_features/WaveletCNN/cwt-tensorflow')
# from cwt import cwtMortlet, cwtRicker,mortletWavelet, rickerWavelet
# tf.compat.v1.enable_eager_execution()


# import pydot

In [0]:
img = tf.placeholder(tf.float32, shape=(None, 784))

In [4]:
# def framing_windowing(signal):
#   pre_emphasis = 0.97
#   frame_size = 256
#   frame_stride = 128
#   nfilt = 20
#   NFFT = 511
#   emphasized_signal = tf.concat([tf.reshape(signal[:,0],[tf.shape(signal)[0],1]),(signal[:,1:] - pre_emphasis * signal[:,:-1])],axis =1)
#   print(emphasized_signal.shape)
#   frame_length, frame_step = frame_size, frame_stride  # Convert from seconds to samples
#   signal_frames=tf.signal.frame(emphasized_signal,frame_length, frame_step)
#   signal_frames *=tf.signal.hamming_window(frame_length)
#   print(signal_frames.shape)
#   return signal_frames
def framing_windowing(signal):
    pre_emphasis = 0.97
    frame_size = 3200
    frame_stride = 160
    nfilt = 20
    emphasized_signal = np.append(signal[0], signal[1:] - pre_emphasis * signal[:-1])
    frame_length, frame_step = frame_size, frame_stride  # Convert from seconds to samples
    signal_length = len(emphasized_signal)
    frame_length = int(round(frame_length))
    frame_step = int(round(frame_step))
    num_frames = int(np.ceil(float(np.abs(signal_length - frame_length)) / frame_step))  # Make sure that we have at least 1 frame
#     print(num_frames)
    pad_signal_length = num_frames * frame_step + frame_length
    z = np.zeros((pad_signal_length - signal_length))
    pad_signal = np.append(emphasized_signal, z) # Pad Signal to make sure that all frames have equal number of samples without truncating any samples from the original signal

    indices = np.tile(np.arange(0, frame_length), (num_frames, 1)) + np.tile(np.arange(0, num_frames * frame_step, frame_step), (frame_length, 1)).T
    frames = pad_signal[indices.astype(np.int32, copy=False)]
    frames *= np.hamming(frame_length)
    # print(frames.shape)
    return frames

In [5]:
# def tkeo_new(a):

#     """
#     Calculates the TKEO of a given recording by using 2 samples.
#     See Li et al., 2007
#     Arguments:
#     a 			--- 1D numpy array.
#     Returns:
#     1D numpy array containing the tkeo per sample
#     """
#     # Create two temporary arrays of equal length, shifted 1 sample to the right
#     # and left and squared:
#     i = a[:,:,1:-1]*a[:,:,1:-1]
#     j = a[:,:,2:]*a[:,:,:-2]
#     # Calculate the difference between the two temporary arrays:
#     aTkeo = i-j
#     return aTkeo
def tkeo(a):

    """
    Calculates the TKEO of a given recording by using 2 samples.
    See Li et al., 2007
    Arguments:
    a 			--- 1D numpy array.
    Returns:
    1D numpy array containing the tkeo per sample
    """
    # Create two temporary arrays of equal length, shifted 1 sample to the right
    # and left and squared:
    i = a[1:-1]*a[1:-1]
    j = a[2:]*a[:-2]
    # Calculate the difference between the two temporary arrays:
    aTkeo = i-j
    return aTkeo

In [4]:
def generator(data, labels, batch_size):
    """
    Yields the next training batch.
    Suppose `samples` is an array [[audio1,label1], [audio2,label2],...].
    """
    

    while True: # Loop forever so the generator never terminates
        X_train,y_train = create_batches_rnd(data, labels, batch_size)
        yield X_train, y_train

In [5]:
def generator_val(data, labels, batch_size):
    """
    Yields the next training batch.
    Suppose `samples` is an array [[audio1,label1], [audio2,label2],...].
    """
    

    while True: # Loop forever so the generator never terminates
        X_train,y_train = create_batches_rnd(data, labels, batch_size)
        yield X_train, y_train

In [5]:
def generator_part1(data, labels,batch_size):
    """
    Yields the next training batch.
    Suppose `samples` is an array [[audio1,label1], [audio2,label2],...].
    """
    x_samples = []
    y_samples = []
    for count,i in enumerate(data):
        x_samples = framing_windowing(i)
        num_samples = len(x_samples)
        y_samples = [labels[count]]*num_samples
#         print("yo")
 
      # Get index to start each batch: [0, batch_size, 2*batch_size, ..., max multiple of batch_size <= num_samples]
        for offset in range(0, num_samples, batch_size):
        # Get the samples you'll use in this batch
            batch_samples = x_samples[offset:offset+batch_size]

        # Initialise X_train and y_train arrays for this batch
            X_train = []
            y_train = []
            wpt = []
        # For each example
            for count,x_sample in enumerate(batch_samples):
            # audio (X) and label (y)
                audio =  x_sample
#             print(level1.shape)
                label = y_samples[count]
            
            # Add example to arrays
                X_train.append(audio)
                y_train.append(label)
                wpt.append(Wavelet_1d(audio))


        # Make sure they're numpy arrays (as opposed to lists)
            X_train = np.array(X_train)
            y_train = np.array(y_train)
#             print(y_train)
#             y_train = to_categorical(y_train)
            wpt = np.array(wpt)
#             print(y_train.shape)
        # The generator-y part: yield the next training batch            
            yield X_train,wpt, y_train

In [6]:
def create_batches_rnd(data,labels,batch_size):
    wlen = 3200
    fact_amp = 0.2
    # Initialization of the minibatch (batch_size,[0=>x_t,1=>x_t+N,1=>random_samp])
    sig_batch=np.zeros([batch_size,wlen])
    wpt_batch = np.zeros([batch_size,wlen,1])
    lab_batch=[]
    signal_id_arr=np.random.randint(data.shape[0], size=batch_size)
    rand_amp_arr = np.random.uniform(1.0-fact_amp,1+fact_amp,batch_size)
    for i in range(batch_size): 
        # select a random sentence from the list 
        #[fs,signal]=scipy.io.wavfile.read(data_folder+wav_lst[snt_id_arr[i]])
        #signal=signal.astype(float)/32768
#         [signal, fs] = sf.read(data_folder+wav_lst[snt_id_arr[i]])
        signal = data[signal_id_arr[i]]
        # accesing to a random chunk
        signal_len=signal.shape[0]
        signal_beg=np.random.randint(signal_len-wlen-1) #randint(0, snt_len-2*wlen-1)
        signal_end=signal_beg+wlen
        sig_batch[i,:]=signal[signal_beg:signal_end]*rand_amp_arr[i]
        wpt = Wavelet_1d(signal[signal_beg:signal_end])
        wpt_batch[i,:] = wpt
        y=labels[signal_id_arr[i]]
#         yt = to_categorical(y, num_classes=out_dim)
        lab_batch.append(y)
    a, b = np.shape(sig_batch)
    sig_batch = sig_batch.reshape((a, b, 1))
    return sig_batch, wpt_batch, np.array(lab_batch)

In [6]:
def create_batches_rnd(data,labels,batch_size):
    wlen = 3200
    fact_amp = 0.2
    # Initialization of the minibatch (batch_size,[0=>x_t,1=>x_t+N,1=>random_samp])
    sig_batch=np.zeros([batch_size,wlen])
#     wpt_batch = np.zeros([batch_size,wlen,1])
    lab_batch=[]
    signal_id_arr=np.random.randint(data.shape[0], size=batch_size)
    rand_amp_arr = np.random.uniform(1.0-fact_amp,1+fact_amp,batch_size)
    for i in range(batch_size): 
        # select a random sentence from the list 
        #[fs,signal]=scipy.io.wavfile.read(data_folder+wav_lst[snt_id_arr[i]])
        #signal=signal.astype(float)/32768
#         [signal, fs] = sf.read(data_folder+wav_lst[snt_id_arr[i]])
        signal = data[signal_id_arr[i]]
        # accesing to a random chunk
        signal_len=signal.shape[0]
        signal_beg=np.random.randint(signal_len-wlen-1) #randint(0, snt_len-2*wlen-1)
        signal_end=signal_beg+wlen
        sig_batch[i,:]=signal[signal_beg:signal_end]*rand_amp_arr[i]
#         wpt = Wavelet_1d(signal[signal_beg:signal_end])
#         wpt_batch[i,:] = wpt
        y=labels[signal_id_arr[i]]
#         yt = to_categorical(y, num_classes=out_dim)
        lab_batch.append(y)
    a, b = np.shape(sig_batch)
    sig_batch = sig_batch.reshape((a, b, 1))
    return sig_batch, np.array(lab_batch)

In [16]:
def generator_val(x_samples, y_samples, batch_size):
    """
    Yields the next training batch.
    Suppose `samples` is an array [[audio1,label1], [audio2,label2],...].
    """
    
    num_samples = len(x_samples)
    while True: # Loop forever so the generator never terminates
        
        # Get index to start each batch: [0, batch_size, 2*batch_size, ..., max multiple of batch_size <= num_samples]
        for offset in range(0, num_samples, batch_size):
            # Get the samples you'll use in this batch
            batch_samples = x_samples[offset:offset+batch_size]

            # Initialise X_train and y_train arrays for this batch
            X_train = []
            y_train = []
            wpt = []
            # For each example
            for count,x_sample in enumerate(batch_samples):
                # audio (X) and label (y)
                audio =  x_sample
#                 print(level1.shape)
                label = y_samples[count]
                
                # Add example to arrays
                X_train.append(audio)
                y_train.append(label)
                wpt.append(Wavelet_1d(audio))


            # Make sure they're numpy arrays (as opposed to lists)
            X_train = np.array(X_train)
            y_train = np.array(y_train)
            y_train = to_categorical(y_train)
            wpt = np.array(wpt)
#             print("hello")
            # The generator-y part: yield the next training batch            
            yield X_train, wpt, y_train

In [6]:
def generator_val_part1(data, labels):
    """
    Yields the next training batch.
    Suppose `samples` is an array [[audio1,label1], [audio2,label2],...].
    """
    x_samples = []
    y_samples = []
    for count,i in enumerate(data):
        x_samples = framing_windowing(i)
        num_samples = len(x_samples)
        y_samples = [labels[count]]*num_samples
        # Get index to start each batch: [0, batch_size, 2*batch_size, ..., max multiple of batch_size <= num_samples]
        
        for offset in range(0, num_samples, batch_size):
            # Get the samples you'll use in this batch
            batch_samples = x_samples[offset:offset+batch_size]

            # Initialise X_train and y_train arrays for this batch
            X_train = []
            y_train = []
            wpt = []
            # For each example
            for count,x_sample in enumerate(batch_samples):
                # audio (X) and label (y)
                audio =  x_sample
#                 print(level1.shape)
                label = y_samples[count]
                
                # Add example to arrays
                X_train.append(audio)
                y_train.append(label)
                wpt.append(Wavelet_1d(audio))


            # Make sure they're numpy arrays (as opposed to lists)
            X_train = np.array(X_train)
            y_train = np.array(y_train)
            wpt = np.array(wpt)
#             print("hello")
            # The generator-y part: yield the next training batch            
            yield X_train, wpt, y_train

In [0]:
# def Wavelet_1d(batch_signal):
#   import numpy
#   pre_emphasis = 0.97
#   frame_size = 256
#   frame_stride = 128
#   nfilt = 20
#   NFFT = 511
#   sample_rate = 16000
#   batch_signal = framing_windowing(batch_signal)
#   batch_signal = tkeo_new(batch_signal)
#   wptree = pywt.WaveletPacket(data=batch_signal.eval(session = sess), wavelet='db1', mode='symmetric')
#   level1 = wptree.get_level(1, order = "freq")
#   level2 = wptree.get_level(2, order = "freq")
#   level3 = wptree.get_level(3, order = "freq")
#   level4 = wptree.get_level(4, order = "freq")
#   # print("level1 data array:",np.array(level1).shape)
#   # print("level2 data array:",np.array(level2).shape)
#   # print("level3 data array:",np.array(level3).shape)
#   # print("level4 data array:",np.array(level4).shape)
#   for node in level1:
#     data_wp = node.data
#     # print("WP data:",np.array(data_wp).shape)
#     # Features group
#     frame_features1.extend(data_wp)
#   mag_frames = numpy.absolute(frame_features1)  # Magnitude of the FFT
#   pow_frames = numpy.abs((mag_frames) ** 2)
#   z = mel.shape[1] - pow_frames.shape[0]
#   pow_frames = np.pad(pow_frames,[(0,z)],'constant', constant_values=0)
#   # print("pow_frames",pow_frames.shape)
#   mel_scaled_features = mel.dot(pow_frames)
#   audio_features1.append(mel_scaled_features)
# #   for count,signal in enumerate(signals):
# #     audio_features1 = []
# #     audio_features2 = []
# #     audio_features3 = []
# #     audio_features4 = []
# #     for f in signal:
# #       tke = tkeo(f)
# #       data_std = StandardScaler().fit_transform(tke.reshape(-1,1)).reshape(1,-1)[0]            
              
# #       #Feature extraction for each node
# #       frame_features1 = []
# #       frame_features2 = []
# #       frame_features3 = []
# #       frame_features4 = []        
      

# #       for node in level2:
# #         data_wp = node.data
# #         # print("WP data:",np.array(data_wp).shape)
# #         # Features group
# #         frame_features2.extend(data_wp)
# #       mag_frames = numpy.absolute(frame_features2)  # Magnitude of the FFT
# #       pow_frames = numpy.abs((mag_frames) ** 2)
# #       mel_scaled_features = mel.dot(pow_frames)
# #       audio_features2.append(mel_scaled_features)
# #       for node in level3:
# #         data_wp = node.data
# #         # print("WP data:",np.array(data_wp).shape)
# #         # Features group
# #         frame_features3.extend(data_wp)
# #       mag_frames = numpy.absolute(frame_features3)  # Magnitude of the FFT
# #       pow_frames = numpy.abs((mag_frames) ** 2)
# #       mel_scaled_features = mel.dot(pow_frames)
# #       audio_features3.append(mel_scaled_features)
# #       for node in level4:
# #         data_wp = node.data
# #         # print("WP data:",np.array(data_wp).shape)
# #         # Features group
# #         frame_features4.extend(data_wp)
# #       mag_frames = numpy.absolute(frame_features4)  # Magnitude of the FFT
# #       pow_frames = numpy.abs((mag_frames) ** 2)
# #       mel_scaled_features = mel.dot(pow_frames)
# #       audio_features4.append(mel_scaled_features)
    
# #     # print("audio_features1:",np.array(audio_features1).shape)
# #     # print("audio_features1:",np.array(audio_features2).shape)
# #     # print("audio_features1:",np.array(audio_features3).shape)
# #     # print("audio_features1:",np.array(audio_features4).shape)
# # #     print("hello")
# #     log_energy1 = numpy.log10(audio_features1)
# #     log_energy1 = pd.DataFrame(log_energy1)
# #     log_energy2 = numpy.log10(audio_features2)
# #     log_energy2 = pd.DataFrame(log_energy2)
# #     log_energy3 = numpy.log10(audio_features3)
# #     log_energy3 = pd.DataFrame(log_energy3)
# #     log_energy4 = numpy.log10(audio_features4)
# #     log_energy4 = pd.DataFrame(log_energy4)
# #     pd.set_option('use_inf_as_null', True)
# #     log_energy1=log_energy1.fillna(log_energy1.mean())
# #     log_energy2=log_energy2.fillna(log_energy2.mean())
# #     log_energy3=log_energy3.fillna(log_energy3.mean())
# #     log_energy4=log_energy4.fillna(log_energy4.mean())
# #     # print("log_energy1: ",log_energy1.shape)
# #     # print("log_energy2: ",log_energy2.shape)
# #     # print("log_energy3: ",log_energy3.shape)
# #     # print("log_energy4: ",log_energy4.shape)
# #     signals_level1[count] = np.array(log_energy1)
# #     signals_level2[count] = np.array(log_energy2)
# #     signals_level3[count] = np.array(log_energy3)
# #     signals_level4[count] = np.array(log_energy4)
# #   # print(signals_level1.shape)
# #   # print(signals_level2.shape)
# #   # print(signals_level3.shape)
# #   # print(signals_level4.shape)  

  return [signals_level1,signals_level2,signals_level3,signals_level4]

def Wavelet_out_shape(input_shapes):
    # print('in to shape')
    return [tuple([None, 590, 20]), tuple([None, 590, 20]), 
            tuple([None, 590, 20]), tuple([None, 590, 20])]

In [10]:
def Wavelet_1d(signal):
    import numpy
    nfilt = 3200
    NFFT = 8191
    sample_rate = 16000
    # signal = tkeo(signal)
    mel = lb.filters.mel(sr=sample_rate, n_fft=NFFT, n_mels=nfilt)
    audio_features = []
    tke = tkeo(signal)
    data_std = StandardScaler().fit_transform(tke.reshape(-1,1)).reshape(1,-1)[0]            
    wptree = pywt.WaveletPacket(data=data_std, wavelet='db1', mode='symmetric')
    level = wptree.maxlevel
    levels = wptree.get_level(level, order = "freq")            
        #Feature extraction for each node
    frame_features = []        
    for node in levels:
        data_wp = node.data
            # Features group
        frame_features.extend(data_wp)
#     print(np.array(frame_features).shape)
    mag_frames = numpy.absolute(frame_features)  # Magnitude of the FFT
    pow_frames = numpy.abs((mag_frames) ** 2)
    mel_scaled_features = mel.dot(pow_frames)
    log_energy = numpy.log10(mel_scaled_features)
    log_energy = pd.DataFrame(log_energy)
    pd.set_option('use_inf_as_null', True)
    log_energy=log_energy.fillna(log_energy.mean())
    return np.array(log_energy)

In [17]:
def Wavelet_1d(signal):
    import numpy
    pre_emphasis = 0.97
    frame_size = 256
    frame_stride = 128
    nfilt = 20
    NFFT = 511
    sample_rate = 16000
    signal = framing_windowing(signal)
    # signal = tkeo(signal)
    mel = lb.filters.mel(sr=sample_rate, n_fft=NFFT, n_mels=nfilt)
    audio_features = []
    for f in signal:
        tke = tkeo(f)
        data_std = StandardScaler().fit_transform(tke.reshape(-1,1)).reshape(1,-1)[0]            
        wptree = pywt.WaveletPacket(data=data_std, wavelet='db1', mode='symmetric')
        level = wptree.maxlevel
        levels = wptree.get_level(level, order = "freq")            
        #Feature extraction for each node
        frame_features = []        
        for node in levels:
            data_wp = node.data
            # Features group
            frame_features.extend(data_wp)
#         print(len(frame_features))
        mag_frames = numpy.absolute(frame_features)  # Magnitude of the FFT
        pow_frames = numpy.abs((mag_frames) ** 2)
        mel_scaled_features = mel.dot(pow_frames)
        audio_features.append(mel_scaled_features)
    
    
#     print("hello")
    log_energy = numpy.log10(audio_features)
    log_energy = pd.DataFrame(log_energy)
    pd.set_option('use_inf_as_null', True)
    log_energy=log_energy.fillna(log_energy.mean())
    return np.array(log_energy)
def Wavelet_out_shape(input_shapes):
    # print('in to shape')
    return [tuple([None, 590, 20]), tuple([None, 590, 20]), 
            tuple([None, 590, 20]), tuple([None, 590, 20])]

In [61]:
Wavelet_1d(X_train[1]).shape

(399, 20)

In [0]:
def Wavelet_1d(signal):
    import numpy
    pre_emphasis = 0.97
    frame_size = 256
    frame_stride = 128
    nfilt = 20
    NFFT = 511
    sample_rate = 16000
    signal = framing_windowing(signal)
    # signal = tkeo(signal)
    mel = lb.filters.mel(sr=sample_rate, n_fft=NFFT, n_mels=nfilt)
    audio_features1 = []
    audio_features2 = []
    audio_features3 = []
    audio_features4 = []
    # print("signal shape",signal.shape)
    for f in signal:
        tke = tkeo(f)
        data_std = StandardScaler().fit_transform(tke.reshape(-1,1)).reshape(1,-1)[0]            
        wptree = pywt.WaveletPacket(data=data_std, wavelet='db1', mode='symmetric')
        level1 = wptree.get_level(1, order = "freq")
        level2 = wptree.get_level(2, order = "freq")
        level3 = wptree.get_level(3, order = "freq")
        level4 = wptree.get_level(4, order = "freq")
        # print("level1 data array:",np.array(level1).shape)
        # print("level2 data array:",np.array(level2).shape)
        # print("level3 data array:",np.array(level3).shape)
        # print("level4 data array:",np.array(level4).shape)          
          #Feature extraction for each node
        frame_features1 = []
        frame_features2 = []
        frame_features3 = []
        frame_features4 = []        
        for node in level1:
            data_wp = node.data
          # print("WP data:",np.array(data_wp).shape)
          # Features group
            frame_features1.extend(data_wp)
        # print("frame_features1",np.array(frame_features1).shape)
        mag_frames = numpy.absolute(frame_features1)  # Magnitude of the FFT
        pow_frames = numpy.abs((mag_frames) ** 2)
        z = mel.shape[1] - pow_frames.shape[0]
        pow_frames = np.pad(pow_frames,[(0,z)],'constant', constant_values=0)
        # print("pow_frames",pow_frames.shape)
        mel_scaled_features = mel.dot(pow_frames)
        audio_features1.append(mel_scaled_features)
        
    
        for node in level2:
            data_wp = node.data
          # print("WP data:",np.array(data_wp).shape)
          # Features group
            frame_features2.extend(data_wp)
        mag_frames = numpy.absolute(frame_features2)  # Magnitude of the FFT
        pow_frames = numpy.abs((mag_frames) ** 2)
        mel_scaled_features = mel.dot(pow_frames)
        audio_features2.append(mel_scaled_features)
        for node in level3:
            data_wp = node.data
          # print("WP data:",np.array(data_wp).shape)
          # Features group
            frame_features3.extend(data_wp)
        mag_frames = numpy.absolute(frame_features3)  # Magnitude of the FFT
        pow_frames = numpy.abs((mag_frames) ** 2)
        mel_scaled_features = mel.dot(pow_frames)
        audio_features3.append(mel_scaled_features)
        for node in level4:
            data_wp = node.data
          # print("WP data:",np.array(data_wp).shape)
          # Features group
            frame_features4.extend(data_wp)
        mag_frames = numpy.absolute(frame_features4)  # Magnitude of the FFT
        pow_frames = numpy.abs((mag_frames) ** 2)
        mel_scaled_features = mel.dot(pow_frames)
        audio_features4.append(mel_scaled_features)
      
    # print("audio_features1:",np.array(audio_features1).shape)
    # print("audio_features1:",np.array(audio_features2).shape)
    # print("audio_features1:",np.array(audio_features3).shape)
    # print("audio_features1:",np.array(audio_features4).shape)
    print("hello")
    log_energy1 = numpy.log10(audio_features1)
    log_energy1 = pd.DataFrame(log_energy1)
    log_energy2 = numpy.log10(audio_features2)
    log_energy2 = pd.DataFrame(log_energy2)
    log_energy3 = numpy.log10(audio_features3)
    log_energy3 = pd.DataFrame(log_energy3)
    log_energy4 = numpy.log10(audio_features4)
    log_energy4 = pd.DataFrame(log_energy4)
    pd.set_option('use_inf_as_na', True)
    log_energy1=log_energy1.fillna(log_energy1.mean())
    log_energy1 = np.array(log_energy1)
    log_energy2=log_energy2.fillna(log_energy2.mean())
    log_energy2 = np.array(log_energy2)
    log_energy3=log_energy3.fillna(log_energy3.mean())
    log_energy3 = np.array(log_energy3)
    log_energy4=log_energy4.fillna(log_energy4.mean())
    log_energy4 = np.array(log_energy4)
    # print("log_energy1: ",log_energy1.shape)
    # print("log_energy2: ",log_energy2.shape)
    # print("log_energy3: ",log_energy3.shape)
    # print("log_energy4: ",log_energy4.shape)
    # signals_level1[count] = np.array(log_energy1)
    # signals_level2[count] = np.array(log_energy2)
    # signals_level3[count] = np.array(log_energy3)
    # signals_level4[count] = np.array(log_energy4)
    # print(signals_level1.shape)
    # print(signals_level2.shape)
    # print(signals_level3.shape)
    # print(signals_level4.shape)  
    
    return log_energy1,log_energy2,log_energy3,log_energy4

def Wavelet_out_shape(input_shapes):
    # print('in to shape')
    return [tuple([None, 590, 20]), tuple([None, 590, 20]), 
            tuple([None, 590, 20]), tuple([None, 590, 20])]

In [17]:
input_shape = None,51221
    
# input_ = Input(input_shape, name='the_input')
input_ = tf.placeholder(tf.float32, shape=input_shape, name= 'the_input')
# wavelet = Lambda(Wavelet_1d, Wavelet_out_shape, name='wavelet')
# tke = wavelet(input_)
input_l1 = tf.placeholder(tf.float32, shape=(None,399,20), name= 'input_l1')
input_l2 = tf.placeholder(tf.float32, shape=(None,399,20), name= 'input_l2')
input_l3 = tf.placeholder(tf.float32, shape=(None,399,20), name= 'input_l3')
input_l4 = tf.placeholder(tf.float32, shape=(None,399,20), name= 'input_l4')
input_1 = Reshape((399,20,1))(input_l1)
input_2 = Reshape((399,20,1))(input_l2)
input_3 = Reshape((399,20,1))(input_l3)
input_4 = Reshape((399,20,1))(input_l4)
print(input_l1.shape)
# print(input_l2)
# print(input_l3)
# print(input_l4)


# level one decomposition starts
print("hi")
conv_1 = Conv2D(64, kernel_size=(3, 3), padding='same', name='conv_1')(input_1)
# #     pool_1 = MaxPooling2D(pool_size=(3,3),padding = 'same')(conv_1)
norm_1 = BatchNormalization(name='norm_1')(conv_1)
relu_1 = Activation('relu', name='relu_1')(norm_1)

# conv_1_2 = Conv2D(64, kernel_size=(3, 3), strides=(2, 2), padding='same', name='conv_1_2')(relu_1)
conv_1_2 = Conv2D(64, kernel_size=(3, 3), padding='same', name='conv_1_2')(relu_1)
#     # pool_1_2 = MaxPooling2D(pool_size=(3,3),padding = 'same')(conv_1_2)
norm_1_2 = BatchNormalization(name='norm_1_2')(conv_1_2)
relu_1_2 = Activation('relu', name='relu_1_2')(norm_1_2)
    

# level two decomposition starts
conv_a = Conv2D(filters=64, kernel_size=(3, 3), padding='same', name='conv_a')(input_2)
#     # pool_a = MaxPooling2D(pool_size=(3,3),padding = 'same')(conv_a)
norm_a = BatchNormalization(name='norm_a')(conv_a)
relu_a = Activation('relu', name='relu_a')(norm_a)
#print(relu_1_2.shape,relu_a.shape)
    
#concate level one and level two decomposition
concate_level_2 = concatenate([relu_1_2, relu_a])
conv_2 = Conv2D(128, kernel_size=(3, 3), padding='same', name='conv_2')(concate_level_2)
# #     pool_2 = MaxPooling2D(pool_size=(3,3),padding = 'same')(conv_2)
norm_2 = BatchNormalization(name='norm_2')(conv_2)
relu_2 = Activation('relu', name='relu_2')(norm_2)

# conv_2_2 = Conv2D(128, kernel_size=(3, 3), strides=(2, 2), padding='same', name='conv_2_2')(relu_2)
conv_2_2 = Conv2D(128, kernel_size=(3, 3), padding='same', name='conv_2_2')(relu_2)
#     # pool_2_2 = MaxPooling2D(pool_size=(3,3),padding = 'same')(conv_2_2)
norm_2_2 = BatchNormalization(name='norm_2_2')(conv_2_2)
relu_2_2 = Activation('relu', name='relu_2_2')(norm_2_2)

#level three decomposition starts 
conv_b = Conv2D(filters=64, kernel_size=(3, 3), padding='same', name='conv_b')(input_3)
#     # pool_b = MaxPooling2D(pool_size=(3,3),padding = 'same')(conv_b)
norm_b = BatchNormalization(name='norm_b')(conv_b)
relu_b = Activation('relu', name='relu_b')(norm_b)

conv_b_2 = Conv2D(128, kernel_size=(3, 3), padding='same', name='conv_b_2')(relu_b)
#     # pool_b_2 = MaxPooling2D(pool_size=(3,3),padding = 'same')(conv_b_2)
norm_b_2 = BatchNormalization(name='norm_b_2')(conv_b_2)
relu_b_2 = Activation('relu', name='relu_b_2')(norm_b_2)
#print(relu_2_2.shape,relu_b_2.shape)
#concate level two and level three decomposition 
concate_level_3 = concatenate([relu_2_2, relu_b_2])
conv_3 = Conv2D(256, kernel_size=(3, 3), padding='same', name='conv_3')(concate_level_3)
# #     pool_3 = MaxPooling2D(pool_size=(3,3),padding = 'same')(conv_3)
norm_3 = BatchNormalization(name='nomr_3')(conv_3)
relu_3 = Activation('relu', name='relu_3')(norm_3)

# conv_3_2 = Conv2D(256, kernel_size=(3, 3), strides=(2, 2), padding='same', name='conv_3_2')(relu_3)
conv_3_2 = Conv2D(256, kernel_size=(3, 3), padding='same', name='conv_3_2')(relu_3)
#     # pool_3_2 = MaxPooling2D(pool_size=(3,3),padding = 'same')(conv_3_2)
norm_3_2 = BatchNormalization(name='norm_3_2')(conv_3_2)
relu_3_2 = Activation('relu', name='relu_3_2')(norm_3_2)

#level four decomposition start
conv_c = Conv2D(64, kernel_size=(3, 3), padding='same', name='conv_c')(input_4)
#     # pool_c = MaxPooling2D(pool_size=(3,3),padding = 'same')(conv_c)
norm_c = BatchNormalization(name='norm_c')(conv_c)
relu_c = Activation('relu', name='relu_c')(norm_c)

conv_c_2 = Conv2D(256, kernel_size=(3, 3), padding='same', name='conv_c_2')(relu_c)
#     # pool_c_2 = MaxPooling2D(pool_size=(3,3),padding = 'same')(conv_c_2)
norm_c_2 = BatchNormalization(name='norm_c_2')(conv_c_2)
relu_c_2 = Activation('relu', name='relu_c_2')(norm_c_2)

conv_c_3 = Conv2D(256, kernel_size=(3, 3), padding='same', name='conv_c_3')(relu_c_2)
#     # pool_c_3 = MaxPooling2D(pool_size=(3,3),padding = 'same')(conv_c_3)
norm_c_3 = BatchNormalization(name='norm_c_3')(conv_c_3)
relu_c_3 = Activation('relu', name='relu_c_3')(norm_c_3)
#print(relu_3_2.shape,relu_c_3.shape)
# concate level level three and level four decomposition
concate_level_4 = concatenate([relu_3_2, relu_c_3])
conv_4 = Conv2D(256, kernel_size=(3, 3), padding='same', name='conv_4')(concate_level_4)
#     # pool_4 = MaxPooling2D(pool_size=(3,3),padding = 'same')(conv_4)
norm_4 = BatchNormalization(name='norm_4')(conv_4)
relu_4 = Activation('relu', name='relu_4')(norm_4)

conv_4_2 = Conv2D(256, kernel_size=(3, 3), strides=(2, 2), padding='same', name='conv_4_2')(relu_4)
#     # pool_4_2 = MaxPooling2D(pool_size=(3,3),padding = 'same')(conv_4_2)
norm_4_2 = BatchNormalization(name='norm_4_2')(conv_4_2)
relu_4_2 = Activation('relu', name='relu_4_2')(norm_4_2)

conv_5_1 = Conv2D(128, kernel_size=(3, 3), padding='same', name='conv_5_1')(relu_4_2)
#     # pool_5_1 = MaxPooling2D(pool_size=(3,3),padding = 'same')(conv_5_1)
norm_5_1 = BatchNormalization(name='norm_5_1')(conv_5_1)
relu_5_1 = Activation('relu', name='relu_5_1')(norm_5_1)

pool_5_1 = AveragePooling2D(pool_size=(3, 3), strides=1, padding='same', name='avg_pool_5_1')(relu_5_1)
flat_5_1 = Flatten(name='flat_5_1')(pool_5_1) 

fc_5 = Dense(2048, name='fc_5')(flat_5_1)
norm_5 = BatchNormalization(name='norm_5')(fc_5)
relu_5 = Activation('relu', name='relu_5')(norm_5)
drop_5 = Dropout(0.5, name='drop_5')(relu_5)

fc_6 = Dense(2048, name='fc_6')(drop_5)
norm_6 = BatchNormalization(name='norm_6')(fc_6)
relu_6 = Activation('relu', name='relu_6')(norm_6)
drop_6 = Dropout(0.5, name='drop_6')(relu_6)

output = Dense(2, activation='softmax', name='fc_7')(drop_6)


(?, 399, 20)
hi


NameError: name 'AveragePooling2D' is not defined

In [14]:
input_shape = None,51221,1
    
# input_ = Input(input_shape, name='the_input')
input_ = tf.placeholder(tf.float32, shape=input_shape, name= 'the_input')
# wavelet = Lambda(Wavelet_1d, Wavelet_out_shape, name='wavelet')
# tke = wavelet(input_)
input_l1 = tf.placeholder(tf.float32, shape=(None,399,20), name= 'input_l1')
input_1 = Reshape((399,20,1))(input_l1)

# level one decomposition starts
print("hi")
conv_1 = Conv2D(64, kernel_size=(3, 3), padding='same', name='conv_1')(input_1)
# #     pool_1 = MaxPooling2D(pool_size=(3,3),padding = 'same')(conv_1)
norm_1 = BatchNormalization(name='norm_1')(conv_1)
relu_1 = Activation('relu', name='relu_1')(norm_1)

# conv_1_2 = Conv2D(64, kernel_size=(3, 3), strides=(2, 2), padding='same', name='conv_1_2')(relu_1)
conv_1_2 = Conv2D(64, kernel_size=(3, 3), padding='same', name='conv_1_2')(relu_1)
#     # pool_1_2 = MaxPooling2D(pool_size=(3,3),padding = 'same')(conv_1_2)
norm_1_2 = BatchNormalization(name='norm_1_2')(conv_1_2)
relu_1_2 = Activation('relu', name='relu_1_2')(norm_1_2)
    

# level two decomposition starts
sinc = sincnet.SincConv1D(20, 251, 16000)(input_)
sinc_pool = MaxPooling1D(pool_size=3,name = 'sinc_pool')(sinc)
sinc_norm = BatchNormalization(momentum=0.05, name = 'sinc_norm')(sinc_pool)
sinc_layer_norm = sincnet.LayerNorm(name = 'sinc_layer_norm')(sinc_norm)
sinc_relu = LeakyReLU(alpha=0.2, name = 'sinc_relu')(sinc_layer_norm)

sinc_conv = Conv1D(20, 251, strides=3, padding='valid')(sinc_layer_norm)
sinc_pool_1 = MaxPooling1D(pool_size=3,name = 'sinc_pool_1')(sinc_conv)
sinc_norm_1 = BatchNormalization(momentum=0.05, name = 'sinc_norm_1')(sinc_pool_1)
sinc_layer_norm_1 = sincnet.LayerNorm(name = 'sinc_layer_norm_1')(sinc_norm_1)
sinc_relu_1 = LeakyReLU(alpha=0.2, name = 'sinc_relu_1')(sinc_layer_norm_1)

sinc_conv_2 = Conv1D(20, 251, strides=3, padding='valid')(sinc_layer_norm)
sinc_pool_2 = MaxPooling1D(pool_size=3,name = 'sinc_pool_1')(sinc_conv)
sinc_norm_ = BatchNormalization(momentum=0.05, name = 'sinc_norm_1')(sinc_pool_1)
sinc_layer_norm_1 = sincnet.LayerNorm(name = 'sinc_layer_norm_1')(sinc_norm_1)
sinc_relu_1 = LeakyReLU(alpha=0.2, name = 'sinc_relu_1')(sinc_layer_norm_1)


#print(relu_1_2.shape,relu_a.shape)
    
#concate level one and level two decomposition
concate_level_2 = concatenate([relu_1_2, sinc_relu_1])
conv_2 = Conv2D(128, kernel_size=(3, 3), padding='same', name='conv_2')(concate_level_2)
# #     pool_2 = MaxPooling2D(pool_size=(3,3),padding = 'same')(conv_2)
norm_2 = BatchNormalization(name='norm_2')(conv_2)
relu_2 = Activation('relu', name='relu_2')(norm_2)

# conv_2_2 = Conv2D(128, kernel_size=(3, 3), strides=(2, 2), padding='same', name='conv_2_2')(relu_2)
conv_2_2 = Conv2D(128, kernel_size=(3, 3), padding='same', name='conv_2_2')(relu_2)
#     # pool_2_2 = MaxPooling2D(pool_size=(3,3),padding = 'same')(conv_2_2)
norm_2_2 = BatchNormalization(name='norm_2_2')(conv_2_2)
relu_2_2 = Activation('relu', name='relu_2_2')(norm_2_2)

#level three decomposition starts 


pool_5_1 = AveragePooling2D(pool_size=(3, 3), strides=1, padding='same', name='avg_pool_5_1')(relu_2_2)
flat_5_1 = Flatten(name='flat_5_1')(pool_5_1) 

fc_5 = Dense(2048, name='fc_5')(flat_5_1)
norm_5 = BatchNormalization(name='norm_5')(fc_5)
relu_5 = Activation('relu', name='relu_5')(norm_5)
drop_5 = Dropout(0.5, name='drop_5')(relu_5)

fc_6 = Dense(2048, name='fc_6')(drop_5)
norm_6 = BatchNormalization(name='norm_6')(fc_6)
relu_6 = Activation('relu', name='relu_6')(norm_6)
drop_6 = Dropout(0.5, name='drop_6')(relu_6)

output = Dense(2, activation='softmax', name='fc_7')(drop_6)


hi


ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concat axis. Got inputs shapes: [(None, 399, 20, 64), (None, 1860, 20)]

In [9]:
input_shape = None,3200,1
    
# input_ = Input(input_shape, name='the_input')
input_ = tf.placeholder(tf.float32, shape=input_shape, name= 'the_input')
# reshaped_input_ =  Reshape((3200,1))(input_)

# wavelet = Lambda(Wavelet_1d, Wavelet_out_shape, name='wavelet')
# tke = wavelet(input_)
input_l1 = tf.placeholder(tf.float32, shape=(None,3200,1), name= 'input_l1')
# reshaped_input_l1 = Reshape((399,20,1))(input_l1)
# reshaped_input_l1 = Reshape((7980,1))(reshaped_input_l1)

# level one decomposition starts
print("hi")
# conv_1 = Conv2D(64, kernel_size=(3, 3), padding='same', name='conv_1')(input_1)
# # #     pool_1 = MaxPooling2D(pool_size=(3,3),padding = 'same')(conv_1)
# norm_1 = BatchNormalization(name='norm_1')(conv_1)
# relu_1 = Activation('relu', name='relu_1')(norm_1)

conv_1 = Conv1D(64, 251, strides=1, padding='valid',name = 'conv_1')(input_l1)
pool_1 = MaxPooling1D(pool_size=3,name = 'pool_1')(conv_1)
norm_1 = BatchNormalization(momentum=0.05, name = 'norm_1')(pool_1)
relu_1 = LeakyReLU(alpha=0.2, name = 'relu_1')(norm_1)


# # conv_1_2 = Conv2D(64, kernel_size=(3, 3), strides=(2, 2), padding='same', name='conv_1_2')(relu_1)
# conv_1_2 = Conv2D(64, kernel_size=(3, 3), padding='same', name='conv_1_2')(relu_1)
# #     # pool_1_2 = MaxPooling2D(pool_size=(3,3),padding = 'same')(conv_1_2)
# norm_1_2 = BatchNormalization(name='norm_1_2')(conv_1_2)
# relu_1_2 = Activation('relu', name='relu_1_2')(norm_1_2)

conv_1_2 = Conv1D(64, 5, strides=2, padding='valid',name = 'conv_1_2')(relu_1)
# pool_1_2 = MaxPooling1D(pool_size=3,name = 'pool_1')(sinc_conv)
norm_1_2 = BatchNormalization(momentum=0.05, name = 'norm_1_2')(conv_1_2)
relu_1_2 = LeakyReLU(alpha=0.2, name = 'relu_1_2')(norm_1_2) 

# paddings = tf.constant([[0, 0],   # the batch size dimension
#                         [261, 260],   # top and bottom of image
#                         [0, 0]])  # the channels dimension
# padded_input = Lambda(lambda x: tf.pad(x, paddings, mode='CONSTANT',
#                       constant_values=0.0))(relu_1_2)

# level two decomposition starts
sinc = sincnet.SincConv1D(64, 251, 16000)(input_)
sinc_pool = MaxPooling1D(pool_size=3,name = 'sinc_pool')(sinc)
sinc_norm = BatchNormalization(momentum=0.05, name = 'sinc_norm')(sinc_pool)
sinc_layer_norm = sincnet.LayerNorm(name = 'sinc_layer_norm')(sinc_norm)
sinc_relu = LeakyReLU(alpha=0.2, name = 'sinc_relu')(sinc_layer_norm)

sinc_conv = Conv1D(64, 5, strides=2, padding='valid')(sinc_layer_norm)
# sinc_pool_1 = MaxPooling1D(pool_size=3,name = 'sinc_pool_1')(sinc_conv)
sinc_norm_1 = BatchNormalization(momentum=0.05, name = 'sinc_norm_1')(sinc_conv)
sinc_layer_norm_1 = sincnet.LayerNorm(name = 'sinc_layer_norm_1')(sinc_norm_1)
sinc_relu_1 = LeakyReLU(alpha=0.2, name = 'sinc_relu_1')(sinc_layer_norm_1)



# sinc_conv_2 = Conv1D(20, 251, strides=1, padding='valid')(sinc_layer_norm_1)
# sinc_pool_2 = MaxPooling1D(pool_size=3,name = 'sinc_pool_2')(sinc_conv_2)
# sinc_norm_2 = BatchNormalization(momentum=0.05, name = 'sinc_norm_2')(sinc_pool_2)
# sinc_layer_norm_2 = sincnet.LayerNorm(name = 'sinc_layer_norm_2')(sinc_norm_2)
# sinc_relu_2 = LeakyReLU(alpha=0.2, name = 'sinc_relu_2')(sinc_layer_norm_2)


#print(relu_1_2.shape,relu_a.shape)
    
#concate level one and level two decomposition
concate_level_2 = concatenate([relu_1_2,sinc_relu_1])
# conv_2 = Conv2D(128, kernel_size=(3, 3), padding='same', name='conv_2')(concate_level_2)
# # #     pool_2 = MaxPooling2D(pool_size=(3,3),padding = 'same')(conv_2)
# norm_2 = BatchNormalization(name='norm_2')(conv_2)
# relu_2 = Activation('relu', name='relu_2')(norm_2)


conv_2 = Conv1D(64, 5, strides=1, padding='valid',name = 'conv_2')(concate_level_2)
# pool_1 = MaxPooling1D(pool_size=3,name = 'pool_2')(sinc_conv)
norm_2 = BatchNormalization(momentum=0.05, name = 'norm_2')(conv_2)
relu_2 = LeakyReLU(alpha=0.2, name = 'relu_2')(norm_2)

# # conv_2_2 = Conv2D(128, kernel_size=(3, 3), strides=(2, 2), padding='same', name='conv_2_2')(relu_2)
# conv_2_2 = Conv2D(128, kernel_size=(3, 3), padding='same', name='conv_2_2')(relu_2)
# #     # pool_2_2 = MaxPooling2D(pool_size=(3,3),padding = 'same')(conv_2_2)
# norm_2_2 = BatchNormalization(name='norm_2_2')(conv_2_2)
# relu_2_2 = Activation('relu', name='relu_2_2')(norm_2_2)

conv_2_2 = Conv1D(64, 5, strides=1, padding='valid',name = 'conv_2_2')(relu_2)
# pool_2_2 = MaxPooling1D(pool_size=3,name = 'pool_1')(sinc_conv)
norm_2_2 = BatchNormalization(momentum=0.05, name = 'norm_2_2')(conv_1_2)
relu_2_2 = LeakyReLU(alpha=0.2, name = 'relu_2_2')(norm_1_2) 

#level three decomposition starts 


pool_5_1 = AveragePooling1D(pool_size=3, padding='same', name='avg_pool_5_1')(relu_2_2)
flat_5_1 = Flatten(name='flat_5_1')(pool_5_1) 

fc_5 = Dense(2048, name='fc_5')(flat_5_1)
norm_5 = BatchNormalization(name='norm_5')(fc_5)
relu_5 = Activation('relu', name='relu_5')(norm_5)
drop_5 = Dropout(0.5, name='drop_5')(relu_5)

fc_6 = Dense(2048, name='fc_6')(drop_5)
norm_6 = BatchNormalization(name='norm_6')(fc_6)
relu_6 = Activation('relu', name='relu_6')(norm_6)
drop_6 = Dropout(0.5, name='drop_6')(relu_6)

output = Dense(2, activation=tf.nn.softmax)(drop_6)


hi


In [7]:
def res_conv_block(X,in_channels,out_channels,stage,block,dilation=1):

    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    X_shortcut = X
    
    X = BatchNormalization(name=bn_name_base+'a')(X)
    X = Activation('relu')(X)
    X = Conv1D(in_channels, 3, padding='valid',use_bias = False, name= conv_name_base+'a')(X)
    X = BatchNormalization(name=bn_name_base+'b')(X)
    X = Activation('relu')(X)
    X = Conv1D(in_channels, 3, padding='valid',use_bias = False, name= conv_name_base+'b')(X)
    print(X.shape)
    paddings = tf.constant([[0, 0],   # the batch size dimension
                          [2, 2],   # top and bottom of image
                          [0, 0]])  # the channels dimension
    X = Lambda(lambda x: tf.pad(x, paddings, mode='CONSTANT',
                        constant_values=0.0))(X)
    X = concatenate([X , X_shortcut])
    X = BatchNormalization(name = bn_name_base+'c')(X)
    X = Activation('relu')(X)
    X = Conv1D(out_channels, 3, padding='valid',use_bias = False, dilation_rate = dilation, name = conv_name_base+'c')(X)

    return X



In [8]:
input_shape = None,3200,1
    

input_ = tf.placeholder(tf.float32, shape=input_shape, name= 'the_input')

# input_l1 = tf.placeholder(tf.float32, shape=(None,3200,1), name= 'input_l1')

# # level one decomposition starts
# print("hi")
# conv_1 = Conv1D(64, 251, strides=1, padding='valid',name = 'conv_1',kernel_initializer = keras.initializers.glorot_uniform(seed=0))(input_l1)
# pool_1 = MaxPooling1D(pool_size=3,name = 'pool_1')(conv_1)
# norm_1 = BatchNormalization(momentum=0.05, name = 'norm_1')(pool_1)
# relu_1 = LeakyReLU(alpha=0.2, name = 'relu_1')(norm_1)


# conv_1_2 = Conv1D(64, 5, strides=2, padding='valid',name = 'conv_1_2',kernel_initializer = keras.initializers.glorot_uniform(seed=0))(relu_1)
# # pool_1_2 = MaxPooling1D(pool_size=3,name = 'pool_1')(sinc_conv)
# norm_1_2 = BatchNormalization(momentum=0.05, name = 'norm_1_2')(conv_1_2)
# relu_1_2 = LeakyReLU(alpha=0.2, name = 'relu_1_2')(norm_1_2) 



# level two decomposition starts
sinc = sincnet.SincConv1D(64, 251, 16000)(input_)
sinc_pool = MaxPooling1D(pool_size=3,name = 'sinc_pool')(sinc)
sinc_norm = BatchNormalization(momentum=0.05, name = 'sinc_norm')(sinc_pool)
sinc_layer_norm = sincnet.LayerNorm(name = 'sinc_layer_norm')(sinc_norm)
sinc_relu = LeakyReLU(alpha=0.2, name = 'sinc_relu')(sinc_layer_norm)

sinc_conv = Conv1D(64, 5, strides=2, padding='valid',kernel_initializer = keras.initializers.glorot_uniform(seed=0))(sinc_layer_norm)
# sinc_pool_1 = MaxPooling1D(pool_size=3,name = 'sinc_pool_1')(sinc_conv)
sinc_norm_1 = BatchNormalization(momentum=0.05, name = 'sinc_norm_1')(sinc_conv)
sinc_layer_norm_1 = sincnet.LayerNorm(name = 'sinc_layer_norm_1')(sinc_norm_1)
sinc_relu_1 = LeakyReLU(alpha=0.2, name = 'sinc_relu_1')(sinc_layer_norm_1)
 
#concate level one and level two decomposition
# concate_level_2 = concatenate([relu_1_2,sinc_relu_1])
# print(concate_level_2.shape)
res_conv_1 = res_conv_block(sinc_relu_1, 128, 16, 1, 'a', 4)
res_conv_2 = res_conv_block(res_conv_1, 16, 8, 2, 'a', 8)
res_conv_3 = res_conv_block(res_conv_2, 8, 4, 3, 'a', 16)
res_conv_4 = res_conv_block(res_conv_3, 4, 2, 4, 'a', 32)
res_conv_5 = res_conv_block(res_conv_4, 2, 1, 5, 'a', 64)

res_norm = BatchNormalization(name='res_norm')(res_conv_5)
res_relu = Activation('relu')(res_norm)





#level three decomposition starts 


pool_5_1 = AveragePooling1D(pool_size=3, padding='same', name='avg_pool_5_1')(res_relu)
flat_5_1 = Flatten(name='flat_5_1')(pool_5_1) 

fc_5 = Dense(2048, name='fc_5',kernel_initializer = keras.initializers.glorot_uniform(seed=0))(flat_5_1)
norm_5 = BatchNormalization(name='norm_5')(fc_5)
relu_5 = Activation('relu', name='relu_5')(norm_5)
drop_5 = Dropout(0.5, name='drop_5')(relu_5)

fc_6 = Dense(2048, name='fc_6',kernel_initializer = keras.initializers.glorot_uniform(seed=0))(drop_5)
norm_6 = BatchNormalization(name='norm_6')(fc_6)
relu_6 = Activation('relu', name='relu_6')(norm_6)
drop_6 = Dropout(0.5, name='drop_6')(relu_6)

output = Dense(2, activation=tf.nn.softmax)(drop_6)


(?, 486, 128)
(?, 478, 16)
(?, 462, 8)
(?, 430, 4)
(?, 366, 2)


In [17]:
!pip install keras==2.2.4


DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
  Using cached Keras-2.2.4-py2.py3-none-any.whl (312 kB)
  Attempting uninstall: keras
    Found existing installation: Keras 2.1.3
    Uninstalling Keras-2.1.3:
      Successfully uninstalled Keras-2.1.3


In [ ]:
total_parameters = 0
for variable in tf.trainable_variables():
    # shape is an array of tf.Dimension
    shape = variable.get_shape()
    print(shape)
    print(len(shape))
    variable_parameters = 1
    for dim in shape:
        print(dim)
        variable_parameters *= dim.value
    print(variable_parameters)
    total_parameters += variable_parameters
print(total_parameters)

In [9]:
def model_summary():
    model_vars = tf.trainable_variables()
    slim.model_analyzer.analyze_vars(model_vars, print_info=True)

model_summary()

---------
Variables: name (type shape) [size]
---------
sinc_conv1d_1/filt_b1:0 (float32_ref 64) [64, bytes: 256]
sinc_conv1d_1/filt_band:0 (float32_ref 64) [64, bytes: 256]
sinc_conv1d_1/Variable:0 (float32_ref 251) [251, bytes: 1004]
sinc_conv1d_1/Variable_1:0 (float32_ref 125) [125, bytes: 500]
sinc_conv1d_1/Variable_2:0 (float32_ref 1) [1, bytes: 4]
sinc_conv1d_1/Variable_3:0 (float32_ref 1) [1, bytes: 4]
sinc_conv1d_1/Variable_4:0 (float32_ref 1) [1, bytes: 4]
sinc_conv1d_1/Variable_5:0 (float32_ref 1) [1, bytes: 4]
sinc_conv1d_1/Variable_6:0 (float32_ref 1) [1, bytes: 4]
sinc_conv1d_1/Variable_7:0 (float32_ref 1) [1, bytes: 4]
sinc_conv1d_1/Variable_8:0 (float32_ref 1) [1, bytes: 4]
sinc_conv1d_1/Variable_9:0 (float32_ref 1) [1, bytes: 4]
sinc_conv1d_1/Variable_10:0 (float32_ref 1) [1, bytes: 4]
sinc_conv1d_1/Variable_11:0 (float32_ref 1) [1, bytes: 4]
sinc_conv1d_1/Variable_12:0 (float32_ref 1) [1, bytes: 4]
sinc_conv1d_1/Variable_13:0 (float32_ref 1) [1, bytes: 4]
sinc_conv1d_1

In [10]:
labels = tf.placeholder(tf.float32, shape=(None,2))
from keras.metrics import categorical_accuracy as accuracy
acc_value = tf.reduce_mean(accuracy(labels, output))
# correct_pred = tf.equal(output, labels)
# accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
from keras.objectives import categorical_crossentropy
loss = tf.reduce_mean(categorical_crossentropy(labels, output))

In [11]:
X_train = np.load("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_train_train.npy")
y_train = np.load("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_train_train_labels.npy")
y_train = to_categorical(y_train)
X_dev_train = np.load("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_dev_train.npy")
X_dev_val = np.load("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_dev_val.npy")
y_dev_train = np.load("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_dev_train_labels.npy")
y_dev_val = np.load("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_dev_val_labels.npy")
# dev_wpt_levels_data_train = np.load("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_dev_wpt_levels_data_train.npy")
# dev_wpt_levels_data_val = np.load("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_dev_wpt_levels_data_val.npy")
# wpt_levels_data_train = np.load("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_train_wpt_levels_data_train.npy")
X_val = np.load("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_train_val.npy")
y_val = np.load("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_train_val_labels.npy")
y_val = to_categorical(y_val)
# wpt_levels_data_val = np.load("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_train_wpt_levels_data_val.npy")

In [12]:
dev_wpt_levels_data_train.shape,dev_wpt_levels_data_val.shape,wpt_levels_data_train.shape,dev_wpt_levels_data_val.shape

NameError: name 'dev_wpt_levels_data_train' is not defined

## Merging dev_train and dev_val with x_train

In [12]:
X_train=np.concatenate([X_train,X_dev_train,X_dev_val])
y_train=np.concatenate([y_train,to_categorical(y_dev_train),to_categorical(y_dev_val)])


In [ ]:
y_train

In [14]:
batch_size = 8
# X_batch = np.array((batch_size,75673))
# y_train = np.array((batch_size))
train_step = tf.train.RMSPropOptimizer(0.001).minimize(loss)

# Initialize all variables
init_op = tf.global_variables_initializer()
sess.run(init_op)
step = 0
# Run training loop
with sess.as_default():
    for i in range(1):
        for X_batch, wpt, y_batch in generator(X_train, y_train, 128):
            step = step + 1
            print(step)
            print(X_batch.shape,wpt.shape,y_batch.shape)
            feed_dict = {input_: X_batch, labels: y_batch, input_l1: wpt, K.learning_phase(): 1}
            sess.run(train_step,feed_dict)
#             print(sess.run(output,feed_dict))
            if step % 1 == 0:
                loss_val,acc_val = (sess.run([loss,acc_value],feed_dict))
                print("Epoch: "+str(i)+"step: "+str(step)+"Training loss: ",loss_val," ","Training accuracy"," ",acc_val)
                loss_val,acc_val = (sess.run([loss,acc_value],feed_dict={input_: X_batch, labels: y_batch, input_l1: wpt, K.learning_phase(): 0}))
                print("Epoch: "+str(i)+"step: "+str(step)+"loss: ",loss_val," ","accuracy"," ",acc_val)
    for X_batch, wpt, y_batch in generator(X_val, y_val, 128):
        loss_val_val,acc_val_val = (sess.run([loss,acc_value],feed_dict={input_: X_batch, labels: y_batch, input_l1: wpt, K.learning_phase(): 0}))
        print("val loss: ",loss_val_val," ","val accuracy"," ",acc_val_val)

1
((128, 3200, 1), (128, 3200, 1), (128, 2))


NameError: name 'input_l1' is not defined

In [ ]:
batch_size = 8
# X_batch = np.array((batch_size,75673))
# y_train = np.array((batch_size))
train_step = tf.train.RMSPropOptimizer(0.001).minimize(loss)

# Initialize all variables
init_op = tf.global_variables_initializer()
sess.run(init_op)
step = 0
# Run training loop
with sess.as_default():
    for i in range(1):
        for X_batch, y_batch in generator(X_train, y_train, 128):
            step = step + 1
            print(step)
            print(X_batch.shape,y_batch.shape)
            feed_dict = {input_: X_batch, labels: y_batch, K.learning_phase(): 1}
            sess.run(train_step,feed_dict)
#             print(sess.run(output,feed_dict))
            loss_val,acc_val = (sess.run([loss,acc_value],feed_dict))
            print("Epoch: "+str(i)+"step: "+str(step)+"Training loss: ",loss_val," ","Training accuracy"," ",acc_val)
            loss_val,acc_val = (sess.run([loss,acc_value],feed_dict={input_: X_batch, labels: y_batch, K.learning_phase(): 0}))
            print("Epoch: "+str(i)+"step: "+str(step)+"loss: ",loss_val," ","accuracy"," ",acc_val)
            if step % 2000 == 0:
                step_val = 0    
                for X_batch, y_batch in generator_val(X_val, y_val, 128):
                    step_val = step_val + 1
                    loss_val_val,acc_val_val = (sess.run([loss,acc_value],feed_dict={input_: X_batch, labels: y_batch, K.learning_phase(): 0}))
                    print("val loss: ",loss_val_val," ","val accuracy"," ",acc_val_val)
                    if step_val == 1000:
                        break

In [11]:
X_train = np.load("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_train_train.npy")
y_train = np.load("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_train_train_labels.npy")
wpt_levels_data_train = np.load("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_train_wpt_levels_data_train.npy")

In [12]:
X_train.shape, wpt_levels_data_train.shape

((3611,), (3611, 4))

In [0]:
from keras.metrics import categorical_accuracy as accuracy

acc_value = accuracy(labels, preds)
with sess.as_default():
    print(acc_value.eval(feed_dict={img: mnist_data.test.images,
                                    labels: mnist_data.test.labels}))

[1. 1. 1. ... 1. 1. 1.]


In [0]:
chunking(X_train,y_train,8)

In [0]:
y_train = np.load("/content/drive/My Drive/SA/Code/spoof_detection_deep_features/WaveletCNN/temp_npy/labels_train.npy",mmap_mode='r')
y_train

memmap([0, 0, 1, ..., 1, 1, 1])